In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda"  # cuda:number or cpu
        self.data = "ctrp"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cuda"

for i, (train_index, test_index) in enumerate(kfold.split(np.arange(pos_num))):
    sampler = Sampler(res, train_index, test_index, null_mask, i)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.810664 auc:0.5082
epoch:  20 loss:0.402165 auc:0.8988
epoch:  40 loss:0.376193 auc:0.9060
epoch:  60 loss:0.369303 auc:0.9072
epoch:  80 loss:0.367265 auc:0.9076
epoch: 100 loss:0.366252 auc:0.9078
epoch: 120 loss:0.365400 auc:0.9081
epoch: 140 loss:0.364352 auc:0.9085
epoch: 160 loss:0.362693 auc:0.9092
epoch: 180 loss:0.359766 auc:0.9104
epoch: 200 loss:0.355636 auc:0.9119
epoch: 220 loss:0.351471 auc:0.9129
epoch: 240 loss:0.347040 auc:0.9138
epoch: 260 loss:0.341122 auc:0.9148
epoch: 280 loss:0.332821 auc:0.9167
epoch: 300 loss:0.328896 auc:0.9182
epoch: 320 loss:0.316522 auc:0.9204
epoch: 340 loss:0.307443 auc:0.9226
epoch: 360 loss:0.300568 auc:0.9241
epoch: 380 loss:0.292585 auc:0.9262
epoch: 400 loss:0.286885 auc:0.9280
epoch: 420 loss:0.284517 auc:0.9290
epoch: 440 loss:0.278155 auc:0.9303
epoch: 460 loss:0.274883 auc:0.9314
epoch: 480 loss:0.271892 auc:0.9324
epoch: 500 loss:0.269021 auc:0.9333
epoch: 520 loss:0.266971 auc:0.9339
epoch: 540 loss:0.264959 auc

In [5]:
true_datas.reset_index(drop=True).to_csv("true_ctrp.csv")
predict_datas.reset_index(drop=True).to_csv("pred_ctrp.csv")